<div align="center">

# ITIS 6120/8120, Applied Databases
## EMR-Dermatology Database System
### Group Members:
#### Anushka Santosh Padyal – 801379909    
#### Nishant Achrekar – 801363902    
#### Shivangi Saxena – 801372350    
#### Bulbul Roy – 801365911
</div>

In [1]:
import mysql.connector
import datetime
import sqlite3
import random
from getpass import getpass

#### print_patients
Lists all the patients present in the database.

When logged in with the user 'admin_staff' and attempts to view all patient data, it gives you output as patient details

In [2]:
def print_patients(conn):
    print('\n--------------------------------------')
    print("Printing all patient details:")
    print('--------------------------------------')
    try:
        cursor = conn.cursor()
        cursor.callproc("getAllPatientNames", args=())
        
        for result in cursor.stored_results():
            print (result.fetchall())

    except mysql.connector.Error as e:
        if "execute command denied" in str(e):
            print(f"Error: {conn.user} is not authorized to perform this action. Access denied.\n")
            
        else:
            print("Error:", e)
    print('--------------------------------------')
        

#### get_patient_by_name
It gets the patient details using the patients first and last name present in the database.

When logged in with the user 'admin_staff' it allows the admin staff to retrieve the patients details using the patients first and last name present in the database.

In [3]:
def get_patient_by_name(conn, name):
    print('\n--------------------------------------')
    print(f"Printing patient details for patient {name}:")
    print('--------------------------------------')
    try:
        name_tokens = name.split(' ', 1)
        cursor = conn.cursor()
        get_patient_info_query = f"SELECT * FROM patient p WHERE p.patient_fname = \'{name_tokens[0]}\' AND p.patient_lname = \'{name_tokens[1]}\'"
        cursor.execute(get_patient_info_query)
        print(cursor.fetchall())

    except mysql.connector.Error as e:
        if "execute command denied" in str(e):
            print(f"Error: {conn.user} is not authorized to perform this action. Access denied.\n")
            
        else:
            print("Error:", e)
    print('--------------------------------------')

#### add_new_patient
It adds a new patient record in the patient and insurance table.

When logged in with the user 'admin_staff' it allows the admin staff to insert/add a new record in the insurance and patient table. We take insurance_id, insurance_provider, fname, lname inputs  for inserting the data into insurance table.

In [4]:
def add_new_patient(conn):
    try:
        print('\n--------------------------------------')
        print("Add new patient")
        print('--------------------------------------')
        patient_id = random.randint(0,65536)
        print(f"New patient ID:{patient_id}")
        fname = input("Patient first name: ")
        lname = input("Patient last name: ")
        gender = input("Patient gender (M/F/O): ")
        phoneNum = input("Patient phone: ")
        email = input("Patient email: ")
        address = input("Patient address: ")
        dob = input("Patient DOB (yyyy-mm-dd): ")
        insurance_id = int(input("Patient insurance number: "))
        insurance_provider = input("Patient insurance provider: ")

        cursor = conn.cursor()
        insert_insurance_query = "INSERT INTO insurance (Insurance_id, In_provider, In_plan_name) VALUES (%s, %s, %s)"
        cursor.execute(insert_insurance_query, (insurance_id, insurance_provider, fname+lname))
        
        cursor = conn.cursor()
        # patient_id, patient_fname, patient_lname, gender, address, phone, email_id, patient_dob ,Insurance_id
        cursor.callproc("PatientInsert", args=(patient_id, fname, lname, gender, address, phoneNum, email, dob, insurance_id))

        #get newly inserted patient information
        get_new_patient_query = f"SELECT * FROM patient p WHERE p.patient_id = {patient_id}"
        cursor.execute(get_new_patient_query)
        print(cursor.fetchone())
        
        cursor.close()
        conn.commit()
        
    except mysql.connector.Error as e:
        if "execute command denied" in str(e):
            print(f"Error: {conn.user} is not authorized to perform this action. Access denied.\n")
            
        else:
            print("Error:", e)
    print('--------------------------------------')

#### update_patient
It shows the updated record from patient table using the patient_id.

When logged in with the user 'admin_staff' it allows the admin staff to fetch and view the patient information from patient table.

In [5]:
def update_patient(conn, id):
    try:
        print('\n--------------------------------------')
        print(f"Updating patient with ID {id}")
        print('--------------------------------------')
        fname = input("Patient first name: ")
        lname = input("Patient last name: ")
        gender = input("Patient gender (M/F/O): ")
        phoneNum = input("Patient phone: ")
        email = input("Patient email: ")
        address = input("Patient address: ")
        dob = input("Patient DOB (yyyy-mm-dd): ")
        
        cursor = conn.cursor()
        # p_patient_id INTEGER, patient_fname VARCHAR(50),patient_lname VARCHAR(50),patient_gender VARCHAR(1), patient_address VARCHAR(100), patient_phone VARCHAR(10),patient_email_id VARCHAR(50),patient_dob date,
        cursor.callproc("patientUpdate", args=(id, fname, lname, gender, address, phoneNum, email, dob))

        #get updated patient information
        print('\n--------------------------------------')
        print(f"Updated patient information with ID {id}:")
        get_updated_patient_query = f"SELECT * FROM patient p WHERE p.patient_id = {id}"
        cursor.execute(get_updated_patient_query)
        print(cursor.fetchone())
        
        cursor.close()
        conn.commit()
        
    except mysql.connector.Error as e:
        if "execute command denied" in str(e):
            print(f"Error: {conn.user} is not authorized to perform this action. Access denied.\n")
            
        else:
            print("Error:", e)
    print('--------------------------------------')

#### delete_patient
It deletes a record from patient table using the patient_id.

When logged in with the user 'admin_staff' it allows the admin staff to delete a single record/row from patient table.
It also takes care of the foreign key constraints and deletes the data from the other tables using the patient_id column.

In [6]:
def delete_patient(conn, id):
    try:
        print('\n--------------------------------------')
        print(f"Deleting patient with ID {id}")
        print('--------------------------------------')
        
        cursor = conn.cursor()
        get_patient_query = f"SELECT * FROM patient p WHERE p.patient_id = {id}"
        result = cursor.execute(get_patient_query)
        patient = cursor.fetchone()
        ip = input(f'Are you sure to delete user {patient[1]} {patient[2]}? (Y/N): ')
        if (ip == 'Y' or ip == 'y'):
            
            delete_medication_query = f"DELETE FROM medication m WHERE m.appointment_id in (SELECT appointment_id FROM appointment a WHERE a.patient_id = {id})"
            result = cursor.execute(delete_medication_query)
            
            delete_payment_query = f"DELETE FROM payment pa WHERE pa.appointment_id in (SELECT appointment_id FROM appointment a WHERE a.patient_id = {id})"
            result = cursor.execute(delete_payment_query)
            
            delete_test_query = f"DELETE FROM patient_test p WHERE p.appointment_id in (SELECT appointment_id FROM appointment a WHERE a.patient_id = {id})"
            result = cursor.execute(delete_test_query)
            
            delete_appointment_query = f"DELETE FROM appointment a WHERE a.patient_id in (SELECT patient_id FROM patient p WHERE p.patient_id = {id})"
            result = cursor.execute(delete_appointment_query)
            
            #Now deleting the patient
            cursor.callproc("delete_patientInfo", args=(id,))

            print('\n--------------------------------------')
            print(f"Deleted patient information with ID {id}:")
            cursor.close()
            conn.commit()
        
    except mysql.connector.Error as e:
        if "execute command denied" in str(e):
            print(f"Error: {conn.user} is not authorized to perform this action. Access denied.\n")
            
        else:
            print("Error:", e)
    print('--------------------------------------')

In [ ]:
print('\n')
print('--------------------------------------')
print("Dermatology Clinic Management System")
print('--------------------------------------')

run = True
while run:
    print('\n\nPlease login to get started. Type \'exit\' to cancel.\n\n')
    user = input("Username: ")
    if user != "exit":
        password = getpass("Password: ")
    else:
        break
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1",
            user=user,
            password=password,
            database="proj_dermatc_db"
        )

        if conn:
            print(f"Welcome {user}!")


            while True:
                print("Please enter your choice:")
                print("1. For printing all patient records")
                print("2. For Adding new patient and insurance record")
                print("3. For updating patient record")
                print("4. For deleting patient record")
                print("5. To Log the user out")
                print("6. To Exit")
                choice = int(input("Your choice: "))

                if choice == 1:
                    print_patients(conn)
                elif choice == 2:
                    add_new_patient(conn)
                elif choice == 3:
                    print_patients(conn)
                    name = input('Type patient name in \'firstname lastname\' form (Like \'John Doe\'):')
                    get_patient_by_name(conn, name)
                    id = input('Type patient ID to update: ')
                    update_patient(conn, id)
                elif choice == 4:
                    print_patients(conn)
                    name = input('Type patient name to delete in \'first name last name\' form (Like \'John Doe\'):')
                    get_patient_by_name(conn, name)
                    id = input('Type patient ID to delete: ')
                    delete_patient(conn, id)
                elif choice == 5:
                    print(f'{user} has been logged out...\n')
                    conn.close()
                    break
                elif choice == 6:
                    run = False
                    print('--------------------------------------')
                    print("Exiting the system - complete")
                    print('--------------------------------------')
                    conn.close()
                    break
                else:
                    print("Invalid choice. Please try again.")
        else:
            print("Invalid credentials! Please try again.")

    except mysql.connector.Error as e:
        if ("Access denied" in str(e)):
            print("Invalid credentials! Please try again.")
        else:
            print("Error:", e)



--------------------------------------
Dermatology Clinic Management System
--------------------------------------


Please login to get started. Type 'exit' to cancel.


Username: test&technical_staff
Password: ········
Welcome test&technical_staff!
Please enter your choice:
1. For printing all patient records
2. For Adding new patient and insurance record
3. For updating patient record
4. For deleting patient record
5. To Log the user out
6. To Exit
Your choice: 1

--------------------------------------
Printing all patient details:
--------------------------------------
Error: test&technical_staff is not authorized to perform this action. Access denied.

--------------------------------------
Please enter your choice:
1. For printing all patient records
2. For Adding new patient and insurance record
3. For updating patient record
4. For deleting patient record
5. To Log the user out
6. To Exit
